# Blog: Unlocking Flexibility with AiiDA-WorkTree




In the realm of AiiDA, workflows have been traditionally orchestrated using two chief components: `workfunction` and `WorkChain`. Both these components come with their distinct attributes and limitations. The `workfunction` shines in simplicity but lacks automatic checkpointing, a crucial feature for long-running calculations. Conversely, `WorkChain` embodies resilience with automatic checkpointing but demands a more intricate implementation and lacks flexibility.


Suppose we want to calculate ```(x + y) * z ``` in two steps. First, add `x` and `y`, then multiply the result with `z`.

#### A example workchain



In [ ]:
%load_ext aiida
from aiida import load_profile
load_profile()
# -*- coding: utf-8 -*-
from aiida.engine import WorkChain, calcfunction
from aiida.orm import Int


@calcfunction
def add(x, y):
    return Int(x + y)

@calcfunction
def multiply(x, y):
    return Int(x * y)

class AddAndMultiplyWorkChain(WorkChain):

    @classmethod
    def define(cls, spec):
        super().define(spec)
        spec.input('x')
        spec.input('y')
        spec.input('z')
        spec.outline(
            cls.add,
            cls.multiply,
            cls.results,
        )
        spec.output('result')

    def add(self):
        self.ctx.sum = add(self.inputs.x, self.inputs.y)

    def multiply(self):
        self.ctx.product = multiply(self.ctx.sum, self.inputs.z)

    def results(self):
        self.out('result', self.ctx.product)



### **Enter AiiDA-WorkTree: The Game-Changer**

Addressing the gaps left by its predecessors, AiiDA-WorkTree introduces a new component: `WorkTree`. This feature simplifies implementation and embraces automatic checkpointing, all while maintaining a high degree of flexibility suitable for designing sophisticated workflows.

In AiiDA, one uses the `calcfunction` to do the calculation and generate new data. AiiDA-WorkTree goes one step further by transforming a `calcfunction` to a `Node`.

In [1]:
from aiida_worktree import node
from aiida.engine import calcfunction

# define add node
@node()
@calcfunction
def add(x, y):
    return x + y

# define multiply node
@node()
@calcfunction
def multiply(x, y):
    return x*y



Profile<uuid='4a9501fb20364cecb99920cadf27b238' name='xing'>

#### Create the workflow
Three steps:

- create a empty `WorkTree`
- add nodes: `add` and `multiply`.
- link the output of the `add` node to one of the `x` input of the `multiply` node.

In [ ]:

from aiida_worktree import WorkTree
from aiida.orm import Int
x = Int(2.0)
y = Int(3.0)
z = Int(4.0)

wt = WorkTree("first_workflow")
wt.nodes.new(add, name="add", x=x, y=y)
wt.nodes.new(multiply, name="multiply", y=z)
wt.links.new(wt.nodes["add"].outputs[0], wt.nodes["multiply"].inputs["x"])

wt.submit(wait=True)


### **Journey Through a Workflow: A Train Analogy**

Imagine planning a train journey. Traditionally, using a WorkChain resembles creating a new train line for every unique journey. Traveling from Bern to Zurich? That's one line (ICE10). Heading from Zurich to Milano? That's another (ICE11). Combining journeys, say from Bern to Milano, would entail the creation of yet another train line (ICE12), despite the redundancy.

However, a more optimized approach exists. A WorkTree allows for a journey where one could travel from Bern to Zurich on ICE10 and then switch to ICE11 from Zurich to Milano. This method eliminates the need for creating a superfluous train line, optimizing resources and improving workflow efficiency.

### **Unpacking the Benefits of WorkTree**

#### **Flexibility in Inputs:**

WorkChains typically require the aggregation of all input parameters at the commencement of the process. Using our train analogy, it is like boarding all passengers in Bern, irrespective of their actual starting point. AiiDA-WorkTree, however, allows for a more nuanced approach, enabling inputs to join at different stages of the workflow.

#### **User-Developer Distinction:**

The advent of WorkTree amplifies the distinction between the AiiDA users and developers. It allows developers to focus on creating foundational WorkChains, likened to establishing essential train routes. Users, on the other hand, can effortlessly utilize and connect these foundational elements to tailor workflows that cater to their specific needs, without delving deep into development.

Certainly! I'll add a new section to your blog that discusses guidelines on when to use WorkTree or WorkChain, considering the nuances of workflow construction and modification needs.


## **Choosing the Right Tool: WorkTree or WorkChain?**

In the world of AiiDA workflows, making an informed choice between WorkTree and WorkChain is pivotal. The decision hinges on the nature of your workflow and specific requirements concerning data provenance and workflow modification.

### **When to Opt for WorkTree?**

WorkTree emerges as a strong candidate when the workflow primarily involves linking outputs of one AiiDA process to the inputs of another, irrespective of the complexity and number of links. WorkTree facilitates this with an emphasis on maintaining comprehensive data provenance.

- **Linkage-Centric Workflows**: If your workflow is a symphony of interconnected processes where outputs and inputs flow seamlessly, WorkTree is your conductor, orchestrating these links with finesse.

- **Preserving Data Provenance**: With WorkTree, each data transformation is meticulously tracked, ensuring a rich and reliable record of data provenance.

### **When is WorkChain the Preferable Choice?**

WorkChain should be your go-to option when your workflow demands numerous slight modifications to the outputs before they metamorphose into inputs for subsequent processes.

- **Modification-Heavy Workflows**: In scenarios where outputs undergo substantial tweaking and adjustments, WorkChain simplifies this by allowing easy modifications through context variables, acting as a flexible black box.

- **Complex Flow Control**: WorkChain shows its mettle when dealing with intricate flow controls. If your workflow dances to the tunes of multiple while loops and conditional statements, WorkChain choreographs this complexity with grace, even if it makes the workflow somewhat elongated and intricate.

### **Striking a Balance**

Choosing between WorkTree and WorkChain is not a binary decision but rather a balancing act. While WorkTree shines in straightforward, linkage-centric workflows with a focus on data provenance, WorkChain takes the stage when the spotlight is on output modifications and complex flow controls. Evaluating the specific needs of your workflow and the intrinsic capabilities of each tool will guide you towards making a choice that resonates with efficiency and effectiveness.

### Execution order
In the scenario of AiiDA WorkChain, where involves a sequence of steps where each step can have parallel jobs, it's typically referred to as Hybrid Sequential-Parallel Execution. This model combines aspects of both sequential and parallel execution:

Sequential Execution at the higher level: The overall process is broken down into distinct steps or stages, where each step must be completed before the next one begins.

Parallel Execution within each step: Within each of these sequential steps, multiple tasks or jobs can be executed in parallel.

This approach is often used in complex workflows where certain tasks are dependent on the completion of all tasks in the previous step, but where there is also room to perform tasks concurrently within each individual step to optimize efficiency and reduce total execution time.

In the case of WorkTree, which involves Dependency-based Execution or a Directed Acyclic Graph (DAG) approach, offers several advantages, especially in complex and multi-step workflows:

Efficient Resource Utilization: By allowing tasks to run as soon as their dependencies are met, this approach can make better use of available resources. It minimizes idle time as tasks don't have to wait for an entire level or sequence to complete.

Increased Parallelism: It naturally enables higher levels of parallelism. Independent tasks can be processed simultaneously, leading to faster overall completion times.

Flexibility and Scalability: The DAG model is highly flexible and scalable. It can accommodate changes in the workflow, such as adding new tasks or modifying dependencies, without major reconfiguration.

Enhanced Error Handling and Debugging: By structuring workflows into smaller, dependent tasks, it's easier to pinpoint and address errors. If a task fails, only its dependent tasks are affected, not the entire workflow.

Improved Workflow Management: It allows for more precise control over the workflow. Tasks can be dynamically scheduled based on resource availability and priority, leading to more efficient workflow management.

Better Clarity and Visualization: DAGs provide a clear visual representation of task dependencies, making it easier to understand and communicate the workflow structure.

Optimized Performance: In scenarios with complex dependencies, DAG-based execution ensures that tasks are executed in an optimal order, leading to better performance and quicker completion times.

Overall, this approach is particularly beneficial in environments where tasks have varying dependencies, and where efficiency and resource optimization are critical.


### **Conclusion: The AiiDA-WorkTree Vision**

AiiDA-WorkTree does not seek to eclipse the existing workflow components like WorkChain, calcfunction, or calcjob. Instead, it envisions enriching the workflow ecosystem, enabling users to craft flexible, resilient, and user-specific workflows with ease. By doing so, AiiDA-WorkTree heralds a new era of workflow management, fostering creativity, efficiency, and precision in the AiiDA community.

## What's Next

|        |                                         |
|---------------|----------------------------------------------------|
| [Concepts](concept/index.rst)                          | A brief introduction of WorkTree’s main concepts.  |
| [Tutorials](tutorial/index.rst)                        | Real-world examples in computational materials science and more.                     |
| [HowTo](howto/index.rst)                              | Advanced topics and tips, e.g flow control using `if`, `for`, `while` and `context`.    |